put in another h1 header explaining that this is the continuation of the previous notebook

process the data thats in the data_processing directory against the terms text thats in that directory. Use the multiprocessing module so that we can gain Nx speed up where N is the number of cpus on a machine. The idea is to
* get all the data from all the retailers we have in the data_processing so far,
* create a large queue with all that data, and create N processes that draw from the Queue !explain that n is the cpus available
* have them each write out a csv with the id of the cpu running it
* when its done combine all those results end to end and that is our result

!mention why we are installing something

In [1]:
pip install pandas

     |████████████████████████████████| 9.3 MB 5.8 MB/s eta 0:00:01     |████████                        | 2.3 MB 5.8 MB/s eta 0:00:02     |████████████▊                   | 3.7 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 50.3 MB/s eta 0:00:01    |████████████████████▌           | 9.3 MB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 47.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
import multiprocessing as mp
import pandas as pd
import datetime

We are now going to collect the data from teh various JSON lines files that we created in the course of performing our scraping

In [45]:
import os
import json
import re
all_text_to_process=[]
for pth,sub,fls in os.walk("./data_processing"):
    for fl in fls:
        # make sure we skip the urls file
        if ".jl" in fl and fl != "all_urls.jl":
            with open(f"{pth}/{fl}","r") as phile:
                ## add each line of the file to the all_text_to_process
                for line in phile:
                    # read the json data on line
                    content = json.loads(line)
                    # add it to the list as a dictionary
                    all_text_to_process.append(content)



Lets create a function that will manipulate our data so that the rest of the code in the notebook has the expected inputs. Each element in `all_text_to_process` is a dictionary with the keys website, text, retailer, but it should also have name, and time. 

In [46]:
def manip_data(e):
    # retrieve the part  of the data that has time information
    # each website has a section /yymmdd...id_/ in it so we will create a regular expression for that
    date_as_string = re.search(r"(\d+)id_",e["website"]).group(1)
    # convert to python date time
    py_date = datetime.datetime.strptime(date_as_string,"%Y%m%d%H%M%S")
    # make it into expected international time standard format string 
    e["time"] = py_date.isoformat(timespec="seconds")
    # make name the same as the retailer key, we do this because other code expects a name key to exist
    e["name"] = e["retailer"]
    # make the website value more understandable and less time specific
    e["website"] = re.search(r"(\d+)id_/(.*)",e["website"]).group(2)

now execute the function on all the elements in our `all_text_to_process`


In [47]:
for e in all_text_to_process:
    manip_data(e)

Now we create a queue out of all the pages we scraped in time

In [53]:
      
## add the items to queue
q = mp.Queue()
for element in all_text_to_process:
    q.put(element)
    
## create a place for the data to go
if not os.path.exists("./data_finished"):
        os.mkdir("./data_finished")
        os.mkdir("./data_finished/uncombined")
        os.mkdir("./data_finished/combined")
        
    


The following function is where the bulk of the work for this notebook gets done. This is where we actually look for keyterms in the website data. Because this is a lot of website data and a lot of terms we will make it parallel by running on all the cpu cores we have on the machine. The function takes in a list of arguments holding its cpu id, and the queue with the elements to process.

The function will add an entry to our `res` list for each webpage we have data for. This helps us track what dates we managed to scrape in visualizations and analysis to come. Without this, we would be left without a clue whether a terms frequency was due to retailers changing the website contents or the wayback machine's scraping schedules. Also the function will add an entry for any of the terms we find in the webpage data.

The final output will be a saved csv in the `./data_finished/uncombined` directory labeled with the cpu core id that the function was running on. Said differently, on a 4 cpu machine you will have 4 files `0.csv` - `3.csv` in the uncombined folder. 

In [54]:

def process_element(args):
    # i is the cpu id
    # qu is the multiprocess queue holding all our website data elements
    i,qu = args

    # load terms and use it as a global variable available to teh multiprocess function process_element
    terms = ""
    with open("./data_processing/terms.txt",'r') as phile:
        terms = phile.read().strip().split("\n")    
    # this is the output result list
    res = []
    ## loop while the queue isn't empty
    while not qu.empty():
        # attempt to pull from the queue
        element = qu.get()
        ## include the scrape record period regardless of term hits, gives context for hits
        res.append(dict(name=element["name"],website=element['website'],term = "NaN",count = "NaN",time = element["time"],term_snippets="NaN"))

        ## go through all the terms and test whether they are in the text

        for term in terms:
            # get occurences of term in website visible text
            count = element["text"].lower().count(term.lower().strip())
            # if we found alteast 1
            if count > 0:
                # get up to 100 characters surrouding the term in the website text, re.findall returns a list
                slim_text = re.findall(" .{0,100}"+ term + ".{0,100} ",element["text"].lower())
                ## perform slight processing, to make a single newline separated string
                snippets ="\n".join([snip.replace("\n","") for snip in slim_text])
                # add to the results
                res.append(dict(name=element["name"],website=element['website'],term = term,count = count,time = element["time"],term_snippets=snippets))
    print("finished queue,saving")
    # create a pandas dataframe for easy csv export from our res list
    df = pd.DataFrame(res)
    # don't include an index column, they get annoying
    df.to_csv(f"./data_finished/uncombined/{i}.csv",index=False)
    

Now that we have defined the function lets set up things for as many different processes running that function as we have cpus.

In [55]:
# get number of cpus, n
cpus = mp.cpu_count()
processes =[]
# create the processes
for i in range(cpus):
    # make actual process with the target being our function, and the id of the cpu (0-n) args the queue from above
    p = mp.Process(target= process_element,args=([i,q],) )
    # start thhe process
    p.start()
    # add it to our list of processes
    processes.append(p)
    
# wait for each to finish
for p in processes:
    p.join()
    
print("done")
## combine the separate files together in the combined directory, add to existing dataset if its there

finished queue,saving
finished queue,saving
finished queue,saving
finished queue,saving
done


Now we will actually put together our final single csv for visualization and analysis.

We will be putting together all the csvs in the uncombined folder, but if we have run this notebook before we should also merge in the final csv from that. This ensures that we don't lose any data if we decide to go through the whole workflow multiple times. We also make sure to de-duplicate our csv data so that we don't see multiple rows with the same retailer, timestamp, and terms.

In [56]:
# make a list containing our pandas dataframes
dfs = []
# look for all the uncombied csvs
for pth,sub,fls in os.walk("./data_finished/uncombined"):
    for fl in fls:
        df = pd.read_csv(f"{pth}/{fl}")
        dfs.append(df)
        

        
        
## see if there's already a complete csv from previous executions of this whole notebook
if os.path.exists("./data_finished/combined/data.csv"):
    df = pd.read_csv("./data_finished/combined/data.csv")
    dfs.append(df)
# take a look at our dataframes in case that's interesting to you?    

print(dfs)

# merge all the dataframes together removing duplicate rows. Note, duplicate means all columns of 2 rows are "exactly the same". 
# if you want to restrict the columns considered for duplication detection look up the details here
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

all_df = pd.concat(dfs).drop_duplicates()

# save the total data to the combined directory with the file name as data.csv, and don't include an index column
all_df.to_csv("./data_finished/combined/data.csv",index=False)
print("csv complete")

[           name                                            website  \
0     fredmeyer                 https://www.fredmeyerjewelers.com/   
1     fredmeyer                 https://www.fredmeyerjewelers.com/   
2     fredmeyer                 https://www.fredmeyerjewelers.com/   
3     fredmeyer                 https://www.fredmeyerjewelers.com/   
4     fredmeyer                 https://www.fredmeyerjewelers.com/   
...         ...                                                ...   
1088   samsclub  https://corporate.samsclub.com/member-update-s...   
1089   samsclub  https://corporate.samsclub.com/member-update-s...   
1090   samsclub  https://corporate.samsclub.com/member-update-s...   
1091   samsclub  https://corporate.samsclub.com/member-update-s...   
1092   samsclub  https://corporate.samsclub.com/member-update-s...   

                    term  count                 time  \
0                    NaN    NaN  2020-09-03T19:39:49   
1                  covid    1.0  2020-09-03T19